# About


In [1]:
import os
import gc  
import time
import pandas as pd
import numpy as np
import geopandas as gpd

import warnings
warnings.simplefilter("ignore")

import rasterio
import rioxarray as rioxr

from joblib import load

from shapely.geometry import mapping
import raster_features as rf

from IPython.display import clear_output

# Assuming repository's parent directory is the home directory
home = os.path.expanduser("~")
os.chdir(os.path.join(home,'iceplant-detection-santa-barbara'))

In [2]:
# whether to save rasters
save_rasters = True
prefix = 'final_model'

# whether to print processing info at runtime
verbose = True
save_processing_times = True
delete_aux_rasters = False

# **************************************************************
clip = True

# **************************************************************
# whether only to process aois
only_aois = True

# **************************************************************
# radius of the disk (in pixels) over which entropy is calculated
entropy_r = 6

# features for snow13
feature_order = ['r', 'r_avg13', 'r_entr13', 
                 'g', 'g_avg13', 'g_entr13', 
                 'b', 'b_avg13', 'b_entr13', 
                 'nir', 'nir_avg13', 'nir_entr13', 
                 'ndvi', 'ndvi_avg13', 'ndvi_entr13', 
                 'month', 'day_in_year']

In [3]:
# length of side of the square window over which average/max/min are calculated.
box_side = entropy_r *2 +1

# **************************************************************
# open shapefile of SB coastal buffer and process it to use it for clipping

coast = gpd.read_file(os.path.join(os.getcwd(), 
                                   'data',
                                   'shapefiles',
                                   'SB_coastal_buffer', 
                                   'SB_coastal_buffer.shp'))
# greate GeoJSON to clip raster with it
coast_geo = coast.geometry.apply(mapping)

# **************************************************************
# load pre-trained random forest classifier
model_fp = os.path.join(os.getcwd(),
                  'code',
                  'B_model_training',
                  'final_model.joblib')
rfc = load(model_fp) 

# **************************************************************

if only_aois:
    scene_ids = ['ca_m_3412037_nw_10_060_20200607',
                 'ca_m_3412039_nw_10_060_20200522']#,
               #  'ca_m_3412040_ne_10_060_20200522',
               #  'ca_m_3411934_sw_11_060_20200521',
               #  'ca_m_3411936_se_11_060_20200521']
else:    
    # select the scene ids from given year that intersect the coastal buffer
    # the itemids of all scenes that intersect the coast were previously stored in a csv
    scene_ids = pd.read_csv(os.path.join(os.getcwd(),
                                         'data',
                                         'NAIP_ids',
                                         'NAIP_2020_SB_coastal_scenes_ids.csv'))
    scene_ids = scene_ids.itemid

# **************************************************************
# prepare folder to save rasters
if save_rasters:
    out_dir = os.path.join(os.getcwd(),
                      'data',
                      'map',
                      'processing_results')
    if os.path.exists(out_dir) == False:
        os.mkdir(out_dir)
    out_dir = os.path.join(out_dir, prefix+'_preds_on_scenes')
    if os.path.exists(out_dir) == False:
        os.mkdir(out_dir)

In [5]:
# ---------------------------------------
# temp directory

temp_dir = os.path.join(os.getcwd(),
                        'code',
                        'C_map_creation',
                        'temp')
if os.path.exists(temp_dir) == False:
    os.mkdir(temp_dir)
# ---------------------------------------
# collect processing information for each scene
times_access = []
times_pre = []
times_features = []
times_class = []
times_post = []
processed = []
reason = []
veg_pixels = [] # number of pixels with ndwi<0.3 and ndwi>0.05
n_pixels = []   # number of non-zero pixels in masked scene

# counter for scenes queued for processing
N = len(scene_ids)

# total time initial point
t_total = time.time()

# ---------------------------------------
# ---------------------------------------

for itemid in scene_ids:    
    
    # ***********************************************************************************************
    # *************************************** DATA ACCESS ****************************************
    # open NAIP scene and clip to coast
    t0 = time.time()
    raster = rf.rioxr_from_itemid(itemid)
    times_access.append(time.time()-t0)

    # *data******************************************************************************************
    # *************************************** PRE-PROCESSING ****************************************
    t0 = time.time()
    if clip:
        #warnings.simplefilter("ignore")
        raster = raster.rio.clip(coast_geo, coast.crs)
        #warnings.simplefilter('default')

    # ---------------------------------------
    # select pixels with data (blacked out portions have 0 on all bands)
    df = rf.raster_as_df(raster.to_numpy(), ['r','g','b','nir'])
    df = df.loc[ (df['nir'] != 0) | (df['r'] != 0) | (df['g'] != 0) | (df['b'] != 0)]
    n_pixels.append(df.shape[0])

    # ---------------------------------------
    # stop if there's no data at intersection
    if df.shape[0] == 0:
        times_pre.append(time.time()-t0)
        rf.finish_processing('no_data', processed, reason, 
                             times_features, times_class, times_post, 
                             veg_pixels, itemid)
        if verbose:
            rf.finish_processing_message('no_data', itemid)

    else:
        # find vegetation pixels to go into model
        # keep indices of water and low-ndvi pixels
        # add ndvi and ndwi features for each pixel
        if verbose:
            print('selected data on scene')
        
        is_veg, water_index, not_veg_index = rf.add_spectral_features(df, 
                                                                      ndwi_thresh = 0.3, 
                                                                      ndvi_thresh = 0.05) 
      # ---------------------------------------
        # stop if there are no vegetation pixels at intersection
        if is_veg.shape[0] == 0:
            times_pre.append(time.time()-t0)            
            rf.finish_processing('no_veg', processed, reason, 
                                 times_features, times_class, times_post, 
                                 veg_pixels, itemid)            
            if verbose:
                rf.finish_processing_message('no_veg', itemid)

        else:
            times_pre.append(time.time()-t0)
            processed.append('Y')
            reason.append('processed')
            if verbose:
                print('selected vegetation pixels')
    # **************************************************************************************
    # ******************************** START CREATING FEATURES *****************************            
            t0 = time.time()          
            # ---------------------------------------
            # discard ndwi and add date features
            is_veg.drop('ndwi', axis=1, inplace=True)
            is_veg = rf.add_date_features(is_veg, 
                                          rf.rioxr_from_itemid(itemid).datetime)

    # *************************************************************************************************
    # ******************************** CREATE R,G,B,NIR AUXILIARY RASTERS *****************************
            # make auxiliary spectral rasters from clipped NAIP 
            band_names = ['r_', 'g_', 'b_', 'nir_']
            tags = ['_avgs', '_entrs']
            window_fps = []
            window_cols = []

            for name, band in zip(band_names,range(1,5)):
                rast_name = name+itemid
                
                for tag in tags:
                    rast_fp = os.path.join(temp_dir, rast_name + tag + '.tif')
                    window_fps.append(rast_fp)        
                    window_cols.append(name.replace('_','')+tag.replace('s',str(box_side)))
                    
                    if os.path.isfile(rast_fp) == False:
                        if tag == '_avgs':
                            rf.avg_raster(raster=raster, band=band, rast_name=rast_name, n=box_side, folder_path=temp_dir)                            
                        elif tag == '_entrs':
                            rf.entropy_raster(raster=raster, band=band, rast_name=rast_name, n=entropy_r, folder_path=temp_dir)
                
            if verbose:
                print('created/verified R,G,B,NIR auxiliary rasters (avgs, entr)')

            # ********************************************************************************************
            # ******************************** CREATE NDVI AUXILIARY RASTERS *****************************
            ndvi = rf.ndvi_xarray(raster)
            band_names.append('ndvi_')
            rast_name = 'ndvi_'+itemid

            for tag in tags:
                rast_fp = os.path.join(temp_dir, rast_name + tag + '.tif')
                window_fps.append(rast_fp)        
                window_cols.append('ndvi' + tag.replace('s',str(box_side)))                

                if os.path.isfile(rast_fp) == False:
                    if tag == '_avgs':
                        rf.avg_raster(rast_data=ndvi, 
                                      crs=raster.rio.crs, 
                                      transf=raster.rio.transform(), 
                                      rast_name=rast_name, 
                                      n=box_side,
                                      folder_path=temp_dir)
                    elif tag == '_entrs':
                        # adjusting to entropy input types
                        ndvi = ndvi*100+100  
                        rf.entropy_raster(rast_data=ndvi.astype('uint8'), 
                                          crs=raster.rio.crs, 
                                          transf=raster.rio.transform(), 
                                          rast_name=rast_name, 
                                          n=entropy_r,
                                          folder_path=temp_dir)
            if verbose:
                print('created/verified NDVI auxiliary rasters (avgs,entr)')
            #free memory
            del ndvi
            gc.collect()

    # *******************************************************************************************
    # *********************** EXTRACT FEATURES FROM AUXILIARY RASTERS ***************************
            window_values = []    
            for fp_aux in window_fps:
                match = rioxr.open_rasterio(fp_aux).squeeze()
                match_vector = match.to_numpy().reshape(match.shape[0]*match.shape[1])
                window_values.append(match_vector)
                if delete_aux_rasters:
                    os.remove(fp_aux)

            df_window = pd.DataFrame(dict(zip( window_cols, window_values)))

            scene_features = pd.concat([is_veg, df_window.iloc[is_veg.index]], axis=1)

            # **********************************************************************************************
            # *************** REMOVE NA VALUES (PIXELS AT EDGE OF CLIPPED PART OF RASTER) ******************
            # combine indices for r_min == 0 and ndvi_min == nan, ndvi_avg == nan

            remove = set()
            for band in [x.replace('s',str(box_side)) for x in ['ndvi'+ y for y in tags]]:
                remove = remove.union(scene_features[band][scene_features[band].isna() == True].index)
            # remove these indices from scene_features
            # no need to add them anywhere else, they will be part of the raster's background
            scene_features = scene_features.drop(remove)

            #free memory            
            del df_window, window_values, match_vector, match, remove
            gc.collect()  

            # ******************************************************************************
            # ******************************** ORDER FEATURES ****************************** 

            scene_features = scene_features[feature_order]
            times_features.append(time.time()-t0)            
            if verbose:
                print('finished assembling features')

            # ***********************************************************************************************
            # *************************************** CLASSIFICATION ****************************************
            t0 = time.time()             
            scene_preds = rfc.predict(np.array(scene_features))

            # ---------------------------------------

            #preds = scene_preds.compute()
            preds = scene_preds
            
            times_class.append(time.time() - t0)            
            if verbose:
                print('finished classification')

            # ************************************************************************************************
            # *************************************** POST-PROCESSING ****************************************
            t0 = time.time()            
            # recover pixel indices for iceplant classifications
            preds_df = pd.DataFrame(preds, 
                                 columns=['is_iceplant'], 
                                 index = scene_features.index)
            is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
            non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

            # ---------------------------------------
            # reconstruct indices into image
            indices = [non_iceplant_index,
                       is_iceplant_index, 
                       not_veg_index,
                       water_index]
            values = [0,    # values assigned to pixels from each index
                      1,
                      2,
                      3]
            reconstruct = rf.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)

            # ---------------------------------------

            times_post.append(time.time() - t0)
            if verbose:
                print('finished post-processing')

            # ************************************************************************************************
            # *************************************** SAVE RASTERS *******************************************  
            if save_rasters:
                filename = prefix+'_preds_' + itemid + '.tif'

                with rasterio.open(
                    os.path.join(out_dir, filename),  # file path
                    'w',           # w = write
                    driver = 'GTiff', # format
                    height = reconstruct.shape[0], 
                    width = reconstruct.shape[1],
                    count = 1,  # number of raster bands in the dataset
                    dtype = rasterio.uint8,
                    crs = raster.rio.crs,
                    transform = raster.rio.transform(),
                ) as dst:
                    dst.write(reconstruct.astype(rasterio.uint8), 1)

            if verbose:
                print('FINISHED: ', itemid)        

    # ***********************************************************************************************
    # ************************************ FINAL INFO MESSAGE ***************************************            
    N = N-1        
    if verbose:
        print('REMAINING: ', N, 'scenes', '\n')
        clear_output(True)


total_time = time.time() - t_total
print('FINISH PROCESSING')
print('TOTAL TIME: ', (total_time)/60, ' mins')

TOTAL TIME:  2.849415071805318  mins


In [9]:
#save times processed and itemids as dataframe
D = { 'itemid': scene_ids,
     'processed': processed,
     'reason':reason,
     'access_times': times_access,
     'pre_times': times_pre,
     'fts_times': times_features,
     'class_times' : times_class,
     'post_times' : times_post, 
     'processed_pix' : n_pixels }
processing_df = pd.DataFrame(D)

if save_processing_times:

    filename = prefix+'_processing_results.csv'

    processing_df.to_csv(os.path.join(out_dir, filename ), index=False)

with open(os.path.join(out_dir,"TOTAL_TIME.txt"), "w") as text_file:
    text_file.write("Total time " + str(total_time))